# Team C. Reinforcement assignment.


# Install needed packages

we will use **keras-rl** as our reinforcement learning library

https://keras-rl.readthedocs.io/en/latest/

**OpenAI gym** as the Atari game proccesor 

https://gym.openai.com/

**pyvirtualdisplay** will help us to finally visualize our training agents. Note that the graph server in colab is not available as we work on remote.

https://pypi.org/project/PyVirtualDisplay/






In [0]:
!pip install keras-rl2

     |████████████████████████████████| 51kB 3.2MB/s 
     |████████████████████████████████| 87.9MB 56kB/s 
     |████████████████████████████████| 501kB 37.6MB/s 
     |████████████████████████████████| 3.1MB 39.9MB/s 
  Created wheel for keras-rl2: filename=keras_rl2-1.0.3-cp36-none-any.whl size=52852 sha256=6d9565e4da41518ad88083eec33b6b3ba39b6d16e7a6d6cc4d8630f30248c636
  Stored in directory: /root/.cache/pip/wheels/75/70/d7/65d0ca0a87c24f988435973065ef7ddf78af43a603a035aac0
Successfully built keras-rl2
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install pyglet
!pip install gym[classic_control]
!pip install pyopengl
!pip install pyvirtualdisplay

In [0]:
import tensorflow as tf
print(tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

2.0.0-beta1


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

# Imports

In [0]:
import argparse
import sys
from PIL import Image
import numpy as np
import gym
import keras.backend as K
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
import easydict # added
from gym import wrappers
from gym.wrappers import Monitor
import math
import glob
import io
import base64

Using TensorFlow backend.


# Initializing Python Virtual Display

In [0]:

from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

# Auxiliar functions for video.

We use Monitor to wrap the environment and show_video function to visualize each episode as a video.

In [0]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    print("showing video: " + mp4)
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Auxiliar funtions for processing the Atari Game images

We resize the images to a given input shape and transform to grayscale

In [0]:
#Standard Atari processing

class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')  # saves storage in experience memory

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

# Testing environment and show_video

In [0]:
env = wrap_env(gym.make("MsPacman-v0"))
observation = env.reset()

while True:
  
    env.render()
    
    action = env.action_space.sample() # taking a random action
   
    observation, reward, done, info = env.step(action) 
   
    if done: 
      break;
            
env.close()
show_video()

showing video: video/openaigym.video.0.126.video000000.mp4


# Environment and args definition

In [0]:
args = easydict.EasyDict({
        "mode": "train",
        "env_name": "MsPacman-v0",
        "weights": None,
})

env = gym.make(args.env_name)
np.random.seed(231)
env.seed(123)
nb_actions = env.action_space.n

# Definition of the convolutional neural net for DQN 

[TODO 1] You need to define a convolutional neural net where the input is 4 frames of 84x84 images and
so many outputs as actions defined in the game (environment)

The convolutional neural net has to be formed by 3 convolutional layers followed by a dense layer, with following parameters

* cv1 --> 32 filters of 8x8 size with strides = 4 and relu activation
* cv2 --> 64 filters of 4x4 size with strides = 2 and relu activation
* cv3 --> 64 filters of 3x3 size with strides = 1 and relu activation
* dense --> 512 units with relu activation

In [0]:
INPUT_SHAPE = (84, 84) # Input size of each image in the game
WINDOW_LENGTH = 4 # number of frames we will consider in each input -- they can be considered as different channels
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import Sequential
img_input = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
if K.image_data_format() == 'channels_last':
    # (width, height, channels)
    model.add(layers.Permute((2, 3, 1), input_shape=img_input))
elif K.image_data_format() == 'channels_first':
    # (channels, width, height)
    model.add(layers.Permute((1, 2, 3), input_shape=img_input))
else:
    raise RuntimeError('Unknown image_dim_ordering.')


#Defining the convolutional neural net

model.add(layers.Convolution2D(32,(8,8),  strides = (4,4), activation = 'relu', input_shape = INPUT_SHAPE)) #cv1
model.add(layers.Convolution2D(64,(4,4), strides = (2,2), activation = 'relu' )) #cv2
model.add(layers.Convolution2D(64,(3,3), strides = (1,1), activation = 'relu')) #cv3
model.add(layers.Flatten()) #flatten
model.add(layers.Dense(512, activation = 'relu')) #dense
model.add(layers.Dense(9, activation = 'softmax')) #output
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1606144   
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 4

# Setting LinearAnnealedPolicy

[TODO 2] Explain the meaning and goal of the function LInearAnnealedPolicy as well as all the arguments
we used

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)

processor = AtariProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=1250000)


# Defining the DQN Agent

In [0]:
from tensorflow.keras.optimizers import Adam
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, 
               target_model_update=10000,
               train_interval=4, delta_clip=1.)

dqn.compile(Adam(lr=.00025), metrics=['mae'])

# Training or Testing depending on the arguments

# Question 4. 1M steps log after - 100000, ordinary DQN

In [0]:
args.mode == 'train'
if args.mode == 'train':

    weights_filename = 'dqn_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn.fit(env, nb_steps=1000000, visualize=False, verbose=2, log_interval=100000)
    #========================
    
elif args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()
    

dqn_MsPacman-v0_weights_{step}.h5f
Training for 1000000 steps ...
    619/1000000: episode: 1, duration: 2.269s, episode steps: 619, steps per second: 273, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.994 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1264/1000000: episode: 2, duration: 2.317s, episode steps: 645, steps per second: 278, episode reward: 18.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 4.064 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1883/1000000: episode: 3, duration: 2.153s, episode steps: 619, steps per second: 288, episode reward: 20.000, mean reward:  0.032 [ 0.000,  1.000], mean action: 4.208 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   2574/1000000: episode: 4, duration: 2.413s, episode steps: 691, steps per second: 286, episode reward: 19.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 3.961 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   317

# Question 5. Testing the DQNAgent using the function test in a single episode.

In [0]:
args.mode = 'test'
if args.mode == 'train':

    weights_filename = 'dqn_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn.fit(env, nb_steps=1000000, visualize=False, verbose=2, nb_max_episode_steps=99, log_interval=100000 )
    #========================
    
elif args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 22.000, steps: 588
showing video: video/openaigym.video.0.146.video000000.mp4


**Notes:**  

We expected the results to be better. However, it seems that packman didn't become smarter since the initial test before learning process. The training took about 7 hours. 

Upon further research we found out that this actually needs to be trained for around 40 million steps to do perfectly  what it is expected to do, which would take weeks to train. Clearly, we don't have that much computing power. https://towardsdatascience.com/advanced-dqns-playing-pac-man-with-deep-reinforcement-learning-3ffbd99e0814  

 Therefore, we decided to augment the model by adding double DQN and dueling with different types. We have decreased the number of steps to 100k to save time. It is interesting to see whether it'll perform better with additional parameters even with less number of steps:



# Question 6. Trying to improve the results obtained from ‘vanilla’ model by using double dqn or/and duel dqn.

*Training only on 100000 steps to save time*


## DQN with 100000 steps

We decided to first run our baseline with 100k steps as well in order to properly gauge the differences in the performance after we alter the model with dueling and double DQN.

In [0]:
# Training
args.mode = 'train'
if args.mode == 'train':

    weights_filename = 'dqn_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000)
    #========================
    
elif args.mode == 'test':
    
    #env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()
    

dqn_MsPacman-v0_weights_{step}.h5f
Training for 100000 steps ...
   619/100000: episode: 1, duration: 2.747s, episode steps: 619, steps per second: 225, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.994 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1274/100000: episode: 2, duration: 2.730s, episode steps: 655, steps per second: 240, episode reward: 16.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 4.072 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1928/100000: episode: 3, duration: 2.685s, episode steps: 654, steps per second: 244, episode reward: 20.000, mean reward:  0.031 [ 0.000,  1.000], mean action: 4.217 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2679/100000: episode: 4, duration: 3.040s, episode steps: 751, steps per second: 247, episode reward: 21.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 4.023 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3294/100000: 

In [0]:
# Testing
args.mode = 'test'
if args.mode == 'train':

    weights_filename = 'dqn_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn.fit(env, nb_steps=1000000, visualize=False, verbose=2, nb_max_episode_steps=99, log_interval=100000 )
    #========================
    
elif args.mode == 'test':
    
    #env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 6.000, steps: 640
showing video: video/openaigym.video.0.245.video000000.mp4


**Notes:**  

As we can see it scored 230 - surprisingly more than with 1 mln steps of training. Now let's alter the model:

## Double DQN agent

In [0]:
from tensorflow.keras.optimizers import Adam
dqn_double = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, 
               target_model_update=10000,
               train_interval=4, delta_clip=1., 
               enable_double_dqn = True)

dqn_double.compile(Adam(lr=.00025), metrics=['mae'])

In [0]:
# Training
args.mode = 'train'
if args.mode == 'train':

    weights_filename = 'dqn_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_double.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_dd' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #========================
    dqn_double.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000)
    #========================
    


dqn_ddMsPacman-v0_weights_{step}.h5f
Training for 100000 steps ...
   619/100000: episode: 1, duration: 1.965s, episode steps: 619, steps per second: 315, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.994 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1264/100000: episode: 2, duration: 1.887s, episode steps: 645, steps per second: 342, episode reward: 18.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 4.062 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1883/100000: episode: 3, duration: 1.715s, episode steps: 619, steps per second: 361, episode reward: 20.000, mean reward:  0.032 [ 0.000,  1.000], mean action: 4.208 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2536/100000: episode: 4, duration: 1.931s, episode steps: 653, steps per second: 338, episode reward: 25.000, mean reward:  0.038 [ 0.000,  1.000], mean action: 3.960 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3299/100000

### Testing double DQN

In [0]:
args.mode = 'test'
if args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn_double.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn_double.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 6.000, steps: 634
showing video: video/openaigym.video.1.1372.video000000.mp4


**Notes:**

As we can see, reward is only 60 - 3.5 times less than our baseline model.

## DQN with dueling 


### Dueling_type = 'avg'

In [0]:
from tensorflow.keras.optimizers import Adam
dqn_duel = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, 
               target_model_update=10000, enable_dueling_network = True, dueling_type = 'avg',
               train_interval=4, delta_clip=1.)

dqn_duel.compile(Adam(lr=.00025), metrics=['mae'])

In [0]:
# Training
args.mode ='train'
if args.mode == 'train':

    weights_filename = 'dqn_duel_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_duel.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_duel_' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_duel_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn_duel.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000)
    #========================

dqn_duel_MsPacman-v0_weights_{step}.h5f
Training for 100000 steps ...
   544/100000: episode: 1, duration: 2.281s, episode steps: 544, steps per second: 239, episode reward: 13.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 4.020 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1552/100000: episode: 2, duration: 4.600s, episode steps: 1008, steps per second: 219, episode reward: 28.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 3.990 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2170/100000: episode: 3, duration: 3.790s, episode steps: 618, steps per second: 163, episode reward: 21.000, mean reward:  0.034 [ 0.000,  1.000], mean action: 4.196 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2764/100000: episode: 4, duration: 3.618s, episode steps: 594, steps per second: 164, episode reward: 21.000, mean reward:  0.035 [ 0.000,  1.000], mean action: 4.146 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3336/10

#### Testing DQN with dueling (avg)

In [0]:
args.mode = 'test'
if args.mode == 'train':

    weights_filename = 'dqn_duel_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_duel.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_duel_' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_duel_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn_duel.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000)
    #========================
    
elif args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn_duel.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn_duel.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 16.000, steps: 608
showing video: video/openaigym.video.1.245.video000000.mp4


**Notes:** 

For 100000 trials the dueling_type = 'avg' did not yield better results than the dqn.

As we can see the model is not so dumb. With more training it might perform way better.
The avg dueling option performed better than double dqn. Final reward was 160 - that is only slightly less than ordinary dqn with 1 mln steps. This has the potential to outperform the baseline model with more steps. Now let's try with different duel type.

### Dueling_type = 'max'


In [0]:
from tensorflow.keras.optimizers import Adam
dqn_duel_max = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, 
               target_model_update=10000, enable_dueling_network = True, dueling_type = 'max',
               train_interval=4, delta_clip=1.)

dqn_duel_max.compile(Adam(lr=.00025), metrics=['mae'])

In [0]:
args.mode = 'train'
if args.mode == 'train':

    weights_filename = 'dqn_duel_max{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_duel_max.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_duel_max' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_duel_max{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn_duel_max.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000)
    #========================

dqn_duel_maxMsPacman-v0_weights_{step}.h5f
Training for 100000 steps ...
   619/100000: episode: 1, duration: 2.603s, episode steps: 619, steps per second: 238, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.994 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1274/100000: episode: 2, duration: 2.833s, episode steps: 655, steps per second: 231, episode reward: 16.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 4.076 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1928/100000: episode: 3, duration: 2.675s, episode steps: 654, steps per second: 244, episode reward: 20.000, mean reward:  0.031 [ 0.000,  1.000], mean action: 4.217 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2535/100000: episode: 4, duration: 2.390s, episode steps: 607, steps per second: 254, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.949 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3238/

#### Testing dueling_type = 'max'

In [0]:
args.mode = 'test'
if args.mode == 'train':

    weights_filename = 'dqn_duel_max{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_duel_max.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_duel_max' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_duel_max{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn_duel_max.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000)
    #========================
    
elif args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn_duel_max.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn_duel_max.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 21.000, steps: 993
showing video: video/openaigym.video.2.1948.video000000.mp4


**Notes:**

The score obtained by dueling type = 'max' is so far the best for 100000 steps - 450. It outperformed 1mln of steps of ordinary DQN.


### Dueling_type = 'naive'

In [0]:
from tensorflow.keras.optimizers import Adam
dqn_duel_naive = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, 
               target_model_update=10000, enable_dueling_network = True, dueling_type = 'naive',
               train_interval=4, delta_clip=1.)

dqn_duel_naive.compile(Adam(lr=.00025), metrics=['mae'])

In [0]:
args.mode ='train'
if args.mode == 'train':

    weights_filename = 'dqn_duel_naive{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_duel_naive.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_duel_naive' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_duel_naive{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn_duel_naive.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000)
    #========================
    

dqn_duel_naiveMsPacman-v0_weights_{step}.h5f
Training for 100000 steps ...
   625/100000: episode: 1, duration: 2.692s, episode steps: 625, steps per second: 232, episode reward: 21.000, mean reward:  0.034 [ 0.000,  1.000], mean action: 3.990 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1260/100000: episode: 2, duration: 3.088s, episode steps: 635, steps per second: 206, episode reward: 23.000, mean reward:  0.036 [ 0.000,  1.000], mean action: 3.986 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1976/100000: episode: 3, duration: 4.418s, episode steps: 716, steps per second: 162, episode reward: 22.000, mean reward:  0.031 [ 0.000,  1.000], mean action: 4.204 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2792/100000: episode: 4, duration: 4.845s, episode steps: 816, steps per second: 168, episode reward: 23.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 4.110 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  344

#### Testing dueling_type = 'naive'

In [0]:
args.mode ='test'
if args.mode == 'train':

    weights_filename = 'dqn_duel_naive{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_duel_naive.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_duel_naive' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_duel_naive{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn_duel_naive.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000)
    #========================
    
elif args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn_duel_naive.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn_duel_naive.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 41.000, steps: 958
showing video: video/openaigym.video.3.1948.video000000.mp4


**Notes:**

The score obtained by dueling type = 'naive' is by far the best score for 100000 steps - 1030. Let's train 300000 steps for dueling_type = 'naive' to see if the results improve further.

### Training dueling_type = 'naive' for 300k steps

In [0]:
args.mode = 'train'
if args.mode == 'train':

    weights_filename = 'dqn_duel_naive{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_duel_naive.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_duel_naive' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_duel_naive{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn_duel_naive.fit(env, nb_steps=300000, visualize=False, verbose=2, log_interval=30000)
    #========================

dqn_duel_naiveMsPacman-v0_weights_{step}.h5f
Training for 300000 steps ...
    619/300000: episode: 1, duration: 3.107s, episode steps: 619, steps per second: 199, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.994 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1264/300000: episode: 2, duration: 3.015s, episode steps: 645, steps per second: 214, episode reward: 18.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 4.064 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   1883/300000: episode: 3, duration: 2.774s, episode steps: 619, steps per second: 223, episode reward: 20.000, mean reward:  0.032 [ 0.000,  1.000], mean action: 4.208 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
   2574/300000: episode: 4, duration: 2.974s, episode steps: 691, steps per second: 232, episode reward: 19.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 3.961 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
 

In [0]:
args.mode = 'test'
if args.mode == 'train':

    weights_filename = 'dqn_duel_naive{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_duel_naive.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_duel_naive' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_duel_naive{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #TODO 3 use fit to train the DQNAgent
    #========================
    dqn_duel_naive.fit(env, nb_steps=300000, visualize=False, verbose=2, log_interval=30000)
    #========================
    
elif args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn_duel_naive.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn_duel_naive.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 40.000, steps: 890
showing video: video/openaigym.video.1.126.video000000.mp4


**Notes:**

Surprisingly training for longer did not improve the results, the same with what we saw when we trained our baseline with 1 mln steps and then 100k steps.

Finally, let's try combining dueling and double DQN and see what results we get.

## Double DQN and Naive dueling together

In [0]:
from tensorflow.keras.optimizers import Adam
dqn_doub_nai = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, 
               target_model_update=10000,
               train_interval=4, delta_clip=1., 
               enable_double_dqn = True, enable_dueling_network = True, dueling_type = 'naive')

dqn_doub_nai.compile(Adam(lr=.00025), metrics=['mae'])

In [0]:
args.mode = 'train'
if args.mode == 'train':

    weights_filename = 'dqn_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_doub_nai.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #========================
    dqn_doub_nai.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000 )
    #========================
    

dqn_MsPacman-v0_weights_{step}.h5f
Training for 100000 steps ...
   619/100000: episode: 1, duration: 1.788s, episode steps: 619, steps per second: 346, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.994 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1274/100000: episode: 2, duration: 1.753s, episode steps: 655, steps per second: 374, episode reward: 16.000, mean reward:  0.024 [ 0.000,  1.000], mean action: 4.076 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1928/100000: episode: 3, duration: 1.733s, episode steps: 654, steps per second: 377, episode reward: 20.000, mean reward:  0.031 [ 0.000,  1.000], mean action: 4.217 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2679/100000: episode: 4, duration: 2.035s, episode steps: 751, steps per second: 369, episode reward: 21.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 4.031 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3294/100000: 

#### Testing double DQN and naive dueling_type

In [0]:
args.mode = 'test'
if args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn_doub_nai.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn_doub_nai.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 8.000, steps: 671
showing video: video/openaigym.video.1.1500.video000000.mp4


**Notes:**

For some reason combining dueling and double DQN got horrible results - the worst so far.

## Double DQN and avg dueling together

In [0]:
from tensorflow.keras.optimizers import Adam
dqn_doub_avg = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, 
               target_model_update=10000,
               train_interval=4, delta_clip=1., 
               enable_double_dqn = True, enable_dueling_network = True, dueling_type = 'avg')

dqn_doub_avg.compile(Adam(lr=.00025), metrics=['mae'])

In [0]:
# Training
args.mode = 'train'
if args.mode == 'train':

    weights_filename = 'dqn_{}_weights.h5f'.format(args.env_name)
    
    if (args.weights): 
      print(args.weights)
      dqn_doub_avg.load_weights(args.weights)
        
    checkpoint_weights_filename = 'dqn_' + args.env_name + '_weights_{step}.h5f'
    print(checkpoint_weights_filename)
    log_filename = 'dqn_{}_log.json'.format(args.env_name)
    callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, 
                                         interval=10000)]
    callbacks += [FileLogger(log_filename, interval=10000)]
    
    
    #========================
    dqn_doub_avg.fit(env, nb_steps=100000, visualize=False, verbose=2, log_interval=10000 )
    #========================
    

dqn_MsPacman-v0_weights_{step}.h5f
Training for 100000 steps ...
   619/100000: episode: 1, duration: 2.090s, episode steps: 619, steps per second: 296, episode reward: 16.000, mean reward:  0.026 [ 0.000,  1.000], mean action: 3.994 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1264/100000: episode: 2, duration: 1.957s, episode steps: 645, steps per second: 330, episode reward: 18.000, mean reward:  0.028 [ 0.000,  1.000], mean action: 4.071 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  1883/100000: episode: 3, duration: 1.932s, episode steps: 619, steps per second: 320, episode reward: 20.000, mean reward:  0.032 [ 0.000,  1.000], mean action: 4.208 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  2574/100000: episode: 4, duration: 2.074s, episode steps: 691, steps per second: 333, episode reward: 19.000, mean reward:  0.027 [ 0.000,  1.000], mean action: 3.975 [0.000, 8.000],  loss: --, mae: --, mean_q: --, mean_eps: --
  3171/100000: 

### Testing double DQN and avg dueling_type

In [0]:
args.mode = 'test'
if args.mode == 'test':
    
    env = wrap_env(gym.make(args.env_name))
    
    if (args.weights):
     dqn_doub_avg.load_weights(args.weights)
    
    #TODO 4 use fit to test the DQNAgent in 1 single episode
    #========================
    dqn_doub_avg.test(env, nb_episodes=1 ,visualize=True)
    #========================
    env.close()
    show_video()

Testing for 1 episodes ...
Episode 1: reward: 53.000, steps: 817
showing video: video/openaigym.video.1.2078.video000000.mp4


**Notes:**

This scored 570 - way better than with naive type but still not the best.

# Conclusions

The best performance in terms of score we got from training DQN Agent with dueling enabled and dueling type 'naive'. Packman scored 1030. It is a bit surprising that increasing the number of steps doesn't seem to improve results, as a matter of fact we saw them deteriorating. Maybe the reason is that the optimal number of steps is around 40 millions, so when increasing them from 100k to 1 million we end up in a local minimum where the model performs worse than with less training. Double DQN didn't seem to increase the performance at all. Combining double DQN and naive dueling scored the worst - maybe for that to do better we needed to alter other hyperparameters. The other reason of double DQN performing worse is that it is supposed to help with overfitting but our number of steps is too small for the model to overfit, thus, it actually ends up severely underfitting. Theoretically, double DQN would perform better with millions of training steps but, again, we don't have enough computing power to test that.